In [0]:
%tensorflow_version 1.x

In [0]:
### In google Colab -- just once: ###
# load into Files:
  # neural.zip
  # data.zip
### run: ###
# !unzip data.zip
# !pip install livelossplot

In [3]:
!unzip -o neural.zip

Archive:  neural.zip
 extracting: neural/__init__.py      
  inflating: neural/architectures.py  
 extracting: neural/core/__init__.py  
  inflating: neural/core/helpers.py  
  inflating: neural/core/training.py  


In [0]:
import pandas as pd
import numpy as np

import helpers_models as hm

from neural.architectures import build_gru
from neural.core.training import full_nn

In [0]:
RUN_SANITY = False
NEW_RANDOM_SPLIT = True

In [0]:
if NEW_RANDOM_SPLIT:
  semilla = np.random.randint(1,9999)
else:
  semilla = 800
rng = np.random.RandomState(semilla)
print(semilla)

In [0]:
datos_raw = hm.read_tagged_data()
datos = hm.clean_tagged_data(datos_raw)
X = datos['text']
y = datos['target']
X_train, X_val, y_train, y_val = hm.split_data(X, y, 0.2, rng)

In [0]:
### GRU sanity check ###
if RUN_SANITY:
  param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
  param_nn = dict(
                        optimizer='adam'
                        ,learn_rate=1e-4
                        ,l2_strength=0.01
                        ,decay_strength=0.0
                        ,momentum=None          
                        ,embedding_dim=512
                  )
  param_train = dict(
                        epochs=5
                        ,batch_size=16
                        ,early_stopping_n=999999
                        ,decay_factor=0.0
                        ,decay_patience_n=999999
                        ,sanity_check_n=20
                        ,verbose=2
                    )
  param_sanity = dict(**param_tokenizer, **param_nn, **param_train)
  mod_check = hn.full_nn(
                hn.build_gru, X_train, y_train, X_val, y_val, **param_sanity
                ) 

In [0]:
# NN parameters
param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
param_nn = dict(
                      optimizer='adam'
                      ,learn_rate=1e-5
                      ,l2_strength=0.01
                      ,decay_strength=0.0
                      ,momentum=None          
                      ,embedding_dim=512
                )
param_train = dict(
                      epochs=30
                      ,batch_size=16
                      ,early_stopping_n=10
                      ,decay_factor=0.9
                      ,decay_patience_n=5
                      ,sanity_check_n=None
                      ,verbose=2
                  )
param = dict(**param_tokenizer, **param_nn, **param_train)

In [0]:
# Training NN
mod = full_nn(build_gru, X_train, y_train, X_val, y_val, **param) 

In [0]:
# TODO: save best model con checkpoint into h5 
# TODO: chequear que f1-prec-rec funcionan OK